# Line Segment Intersection

**Problem**: given a set $S$ of $n$ closed segments in the plane, report all intersection points among the segments in $S$.

Naively, we can simply take each pair of segments, compute whether they intersect and, if so, report their intersection point. This brute-force algorithm takes $O(n^2)$ time. This algorithm is a good approach when each pair of segments in $S$ intersects, every algorithm takes at least $\Omega(n^2)$ time, because it has to report all intersections. In practical situations, most segments intersect no or only a few other segments, so the total number of intersection points is much smaller than quadratic. We want an algorithm whose running time depends not only on the number of segments in the input, but also on the number of intersection points in the output. This kind of algorithm is called _output-sensitive algorithm_. We will see how we can achieve this in the next section.

Let $S := \{s_1, s_2, \dots, s_n\}$ be the set of segments for which we want to compute all intersections. We want to avoid testing pairs of segments that are far apart. Define the $y$-interval of a segment to be its orthogonal projection onto the $y$-axis. When the $y$-intervals of pairs of segments do not overlap - we could say that they are far apart in the $y$-direction and they cannot intersect. Hence, we only need to test pairs of segments whose $y$-intervals overlap, that is, pairs for which there exists a horizontal line that intersects both segments. To find these pairs, we imagine sweeping a line $l$ downwards over the plane, starting from a position above all segments. While we sweep the imaginary line, we keep track of all segments intersecting it.

This type of algorith is called _plane sweep algorithm_ and the line $l$ is called the _sweep line_. The _status_ of the sweep line is the set of segments intersecting it. The status changes while the sweep line moves downwards, but not continuously. Only at particular points is an update of the status required. We call these points the _event points_ of the plane sweep algorithm. In this algorithm, the event points are the endpoints of the segments.

The moments at which the sweep line reaches an event point are the only moments when the algorithm actually does something: it updates the status of the sweep line and performs some intersection tests. In particular, if the event point is the upper endpoint of a segment, then a new segment starts intersecting the sweep line and must be added to the status. This segment is tested for intersection against the ones already intersecting the sweep line. If the event point is a lower endpoint, a segment stops intersecting the sweep line and must be deleted from the status. This way, we only test pairs of segments for which there is a horizontal line that intersects both segments. Unfortunately, this is not enough: there are still situations where we test a quadratic number of pairs, whereas there is only a small number of intersection points. A simple example is a set of vertical segments that all intersect the $x$-axis. So this algorithm is not output-sensitive. The problem is that two segments that intersect the sweep line can still be far apart in the horizontal direction.

Let's order the segments from left to right as they intersect the sweep line, to include the idea of being close in the horizontal direction. We shall only test segments when they are adjacent in the horizontal ordering. This means that we only test any new segment against two segments, namely, the ones immediately left and right of the upper endpoint. Later, when the sweep line has moved downwards to another position, a segment can become adjacent to other segments against which it will be tested. The status now corresponds to the _ordered_ sequence of segments intersecting the sweep line. The new status not only changes at endpoints of segments, it also changes at intersection points, where the order of the intersected segments changes. When this happens, we must test the two segments that change position against their new neighbors. This is a new type of event point.

Now we need to test if this approach is correct by asking ourselves will we find all intersections? In other words, if two segments $s_i$ and $s_j$ intersect, is there always a position of a sweep line $l$ where $s_i$ and $s_j$ are adjacent along $l$?. The next lemma proves this statement:

**Lemma 1** - Let $s_i$ and $s_j$ be two non-horizontal segments whose interiors intersect in a single point $p$, and assume there is no third segment passing through $p$. Then there is an event point above $p$ where $s_i$ and $s_j$ become adjacent and are tested for intersection.

*Proof* - Let $l$ be a horizontal line slightly above $p$. If $l$ is close enough to $p$, then $s_i$ and $s_j$ must be adjacent along $l$. In other words, there is a position of the sweep line where $s_i$ and $s_j$ are adjacent. On the other hand, $s_i$ and $s_j$ are not yet adjacent when the algorithm starts, because the sweep line starts above all line segments and the status is empty. Hence, there must be an event point $q$ where $s_i$ and $s_j$ become adjacent and are tested for intersection.

After we have swept the whole plane, we have computed all intersection points. The invariant of this algorithm is that above the sweep line $l$, we have correctly computed all intersection points.

There are some degenerate cases with this approach, like three or more segments meeting in a point and two partially overlapping segments. We will see how we can solve for three or more segments meeting in a point and ignore, for now, partially overlapping segments.

We start by describing the data structures the algorithm uses:

_Event queue_ is a data structure that stores the events. We denote the event queue $Q$. We need an operation that removes the next event that will occur from $Q$ and returns it so it can be treated. This event is the highest event below the sweep line. If two event points have the same $y$-coordinate, then one with smaller $x$-coordinate will be returned. This implies that we should consider the left endpoint of a horizontal segment to be its upper endpoint and its right endpoint to be its lower endpoint. The event queue must allow insertions, because new events will be computed on the fly. Two event points can coincide. It is convenient to treat this as one event point. Hence, an insertion must be able to check whether an event is already present in $Q$.

We implement the event queue as follows: Define an order $\prec$ on the event points that represents the order in which they will be handled. If $p$ and $q$ are two event points, then we have $p \prec q$ if and only if $p_y > q_y$ holds or $p_y = q_y$ and $p_x < q_x$ holds. We store the event points in a balanced binary search tree, ordered according to $\prec$. With each event point $p$ in $Q$ we will store the segments starting at $p$, that is, the segments whose upper endpoint is $p$. This information will be needed to handle the event. Fetching the next event point and inserting an event take $O(logm)$ time, where $m$ is the number of events in $Q$.

We also need to maintain the status of the algorithm. This is the ordered sequence of segments intersecting the sweep line. The status structure, denoted by $\mathcal{T}$, is used to access the neighbors of a given segment $s$, so that they can be tested for intersection with $s$. The status structure must be dynamic: as segments start or stop intersecting the sweep line, they must be inserted into or deleted from the structure. Because there is a well-defined order on the segments in the status structure, we can use balanced binary search tree as status structure. 

We store the segments intersecting the sweep line ordered in the leaves of a balanced binary search tree $\mathcal{T}$. The left-to-right order of the segments along the sweep line corresponds to the left-to-right order of the leaves in $\mathcal{T}$. We must also store information in the internal nodes to guide the search down the tree to the leaves. At each internal node, we store the segment from the rightmost leaf in its left subtree. Suppose we search in $\mathcal{T}$ for the segment immediately to the left of some point $p$ that lies on the sweep line. At each internal node $v$ we test whether $p$ lies left or right of the segment stored at $v$. Depending on the outcome, we descend to the left or right subtree of $v$, eventually ending up in a leaf. Either this leaf, or the leaf immediately to the left of it, stores the segment we are searching for. In a similar way we can find the segment immediately to the right of $p$, or segments containing $p$. It follows that each update and neighbor search operation takes $O(logn)$ time.

Now that we have described the algorithm and introduced data structures, we now present the pseudo-code of FindIntersections algorithm:

**Algorithm** FindIntersections($S$) \
_Input_ A set $S$ of line segments in the plane\
_Output_ The set of intersection points among the segments in $S$, with for each interesction point the segments that contain it.
1. Initialize an empty event queue $Q$. Next, insert the segment endpoints into $Q$; when an upper endpoint is inserted, the corresponding segment should be stored with it
2. Initialize an empty status structure $\mathcal{T}$
3. **while** $Q$ is not empty
4. &emsp;**do** Determine the next event point $p$ in $Q$ and delete it.
5. &emsp;&emsp; HandleEventPoint($p$)

At endpoints of segment we have to insert or delete segments from the status structure $\mathcal{T}$ and at intersection point we have to change the order of two segments. In both cases, we also have to do intersection tests between segments that become neighbors after the event. In degenerate cases where several segments are involved in one event point is a bit tricky to handle. We describe now the HandleEventPoint procedure:

HandleEventPoint($p$)
1. Let $U(p)$ be the set of segments whose upper endpoint is $p$; these segments are stored with the event point $p$
2. Find all segments stored in $\mathcal{T}$ that contain $p$; they are adjacent in $\mathcal{T}$ Let $L(p)$ denote the subset of segments found whose lower endpoint is $p$, and let $C(p)$ denote the subset of segments found that contain $p$ in their interior.
3. **if** $L(p) \cup U(p) \cup C(p)$ contains more than one segment
4. &emsp; **then** Report $p$ as an intersection, together with $L(p), U(p)$, and $C(p)$.
5. Delete the segments in $L(p) \cup C(p)$ from $\mathcal{T}$.
6. Insert the segments in $U(p) \cup C(p)$ into $\mathcal{T}$. The order of the segments in $\mathcal{T}$ should correspond to the order in which they are intersected by a sweep line just below $p$. If there is a horizontal segment, it comes last among all segments containing $p$.
7. (\*Deleting and reinserting the segments of $C(p)$ reverses their order\*)
8. **if** $U(p) \cup C(P) = \emptyset$
9. &emsp; **then** Let $s_l$ and $s_r$ be the left and right neighbors of $p$ in $\mathcal{T}$
10. &emsp;&emsp; FindNewEvent($s_l, s_r, p)$
11. &emsp; **else** Let $s'$ be the leftmost segment of $U(p) \cup C(p)$ in $\mathcal{T}$.
12. &emsp;&emsp; Let $s_l$ be the left neighbor of $s'$ in $\mathcal{T}$.
13. &emsp;&emsp; FindNewEvent($s_l, s', p$)
14. &emsp;&emsp; Let $s''$ be the rightmost segment of $U(p) \cup C(p)$ in $\mathcal{T}$.
15. &emsp;&emsp; Let $s_r$ be the right neighbor of $s''$ in $\mathcal{T}$.
16. &emsp;&emsp; FindNewEvent($s'', s_r, p$)

**Note**: We assume that $s_l$ and $s_r$ actually exist. If they do not exist, the corresponding steps should obviously not be performed.

The FindNewEvent is defined as follows:

FindNewEvent($s_l, s_r, p$)
1. **if** $s_l$ intersect below the sweep line, or on it and to the right of the current event point $p$, and the intersection is not yet present as an event in $Q$
2. &emsp; **then** Insert the intersection point as an event into $Q$.

Now we need to check the correctness of FindIntersections. The next lemma confirms it:

**Lemma 2** - Algorithm FindIntersections computes all intersection points and the segments that contain it correctly.

*Proof* -

We now have proven that the algorithm is correct. The next lemma proves the running time of the algoritm is in $O((n + I)logn)$, where $I$ is the number of intersections:

**Lemma 3** - The running time of Algorithm FindIntersections for a set $S$ of $n$ line segments in the plane is $O(nlogn + Ilogn)$, where I is the number of intersection points of segments in $S$.

*Proof* -

We still have to analyze the other complexity aspect, the amount of storage used by the algorithm. The tree $\mathcal{T}$ stores a segment at most once, so it uses $O(n)$ storage. The size of $Q$ can be larger, however. The algorithm  inserts intersection points in $Q$ when they are detected and it removes them when they are handled. When it takes a long time before intersections are handled, it could happen that $Q$ gets very large. but it is bounded by $O(n + I)$.

There is a way to achieve to get the working storage of $Q$ to be linear - only store intersection points of pairs of segments that are currently adjacent on the sweep line. The algorithm given above also stores intersection points of segments that have been horizontally adjacent, but aren't anymore. By storing only intersection points among adjacent segments, the number of event points in $Q$ is never more than linear. The modification required in the algorithm is that the intersection point of two segments must be deleted when they stop being adjacent again before the intersection point is reached, so the intersection point will still be reported correctly. The total time taken by the algorithm remains $O(nlogn + Ilogn)$. We now obtain the following theorem:

**Theorem 4** - Let $S$ be a set of $n$ line segments in the plane. All intersection points in $S$, with for each intersection point the segments involved in it, can be reported in $O(nlogn + Ilogn)$ time and $O(n)$ space, where $I$ is the number of intersection points. 

## Theory

- Line Segment Intersection problem in geometric setting
- Intersection specification
- Brute force algorithm for computing line segment intersections
- Definition of output-sensitive algorithm
- Plane sweep algorithm explanation
- Status and end points of the algorithm
- Ordered sequence of lines intersecting the sweep line
- Correctness of the plane sweep with reducing the complexity
- Lemma - Existence of an event point to test two segments for intersection
- Types of event points
- Degenerate cases description
- Data Structures that plane sweep for line segment intersection algorithm uses
    - event queue
    - status structure
- FindIntersections Algorithm pseudo-code
- Correctness of FindIntersections
- Running time of the FindIntersections
- Memory space complexity

## Implementation

- Event Queue
- Status Structure
- FindIntersectionAlgorithm
- HandleEventPoint
- FindNewEventPoint